In [4]:
project = "bigquery-public-data"
dataset_id = "iowa_liquor_sales"
table_id = 'sales'

In [7]:
# get schema

client = bigquery.Client()

dataset_ref = client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)

f = io.StringIO("")
client.schema_to_json(table.schema, f)
schema = f.getvalue()
print(schema)

[
  {
    "description": "Concatenated invoice and line number associated with the liquor order. This provides a unique identifier for the individual liquor products included in the store order.",
    "mode": "NULLABLE",
    "name": "invoice_and_item_number",
    "type": "STRING"
  },
  {
    "description": "Date of order",
    "mode": "NULLABLE",
    "name": "date",
    "type": "DATE"
  },
  {
    "description": "Unique number assigned to the store who ordered the liquor.",
    "mode": "NULLABLE",
    "name": "store_number",
    "type": "STRING"
  },
  {
    "description": "Name of store who ordered the liquor.",
    "mode": "NULLABLE",
    "name": "store_name",
    "type": "STRING"
  },
  {
    "description": "Address of store who ordered the liquor.",
    "mode": "NULLABLE",
    "name": "address",
    "type": "STRING"
  },
  {
    "description": "City where the store who ordered the liquor is located",
    "mode": "NULLABLE",
    "name": "city",
    "type": "STRING"
  },
  {
    "de

In [58]:
table_name = project+"."+dataset_id+"."+table_id
question = "how many average bottles per day were sold in the last month?"

In [59]:
import vertexai
from vertexai.language_models import CodeGenerationModel

vertexai.init(project="felipe-sandbox-354619", location="us-central1")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024
}

model = CodeGenerationModel.from_pretrained("code-bison@001")
prompt = schema + "\n\n based on the schema above from project" + project + "dataset id" + dataset_id + "and table id" + table_id + " can you write me a sql query that does answers the following question: " + question
response = model.predict(
    prefix = prompt,
    **parameters
)
query = response.text.replace("```", "").replace("sql\n","")
print(f"Response from Model: {query}")


Response from Model: SELECT AVG(bottles_sold)
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE date BETWEEN CURRENT_DATE - 30 AND CURRENT_DATE



In [60]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()
job = client.query(query)
df = job.to_dataframe()

In [61]:
df

,f0_
0,12.227503
